<a href="https://colab.research.google.com/github/asabbah44/Text_Classification/blob/main/BertCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece
# !pip install ktrain

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/MydatasetV5.csv')


df1 = pd.DataFrame(df1, columns = ['commenttext','label'])
df2 = pd.DataFrame(df2, columns = ['commenttext','label'])

df=df2 #pd.concat([df1,df2])



In [ ]:
df.shape

(1513, 2)

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
comments= []
sentences = list(df['commenttext'])
for sen in sentences:
    comments.append(preprocess_text(sen))

In [ ]:
df.label.unique()

array(['Documentation', 'Requirement', 'Test', 'Design', 'Defect'],
      dtype=object)

In [ ]:
y = df['label']

y = np.array(list(map(lambda x: 0 if x=="Requirement" else 1 if x=="Design" else 2 if x=="Defect" else 3 if x=="Test" else 4, y)))

Creating a BERT Tokenizer

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenizer.tokenize("don't need this classificatio")

['don', "'", 't', 'need', 'this', 'class', '##ific', '##ati', '##o']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("don't need this class"))

[2123, 1005, 1056, 2342, 2023, 2465]

In [ ]:
def tokenize_comments(text_comments):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_comments))

In [ ]:
tokenize_comments = [tokenize_comments(comment) for comment in comments]

Prerparing Data For Training


In [ ]:
comments_with_len = [[comment, y[i], len(comment)]
                 for i, comment in enumerate(tokenize_comments)]

In [ ]:
print(comments_with_len)

[[[5587, 2261, 5754, 17287, 9285, 1999, 1996, 11867, 2072, 5754, 17287, 9285, 2622, 2008, 2057, 2064, 2224, 2000, 5754, 17287, 2618, 2203, 26521, 2061, 2008, 2057, 2064, 8073, 9699, 16129, 12653, 2005, 2129, 2000, 9530, 8873, 27390, 2063, 2068, 2738, 2066, 5003, 8159, 3289, 2147, 2156, 19130, 2005, 2062, 4751, 1996, 5754, 17287, 9285, 2024, 1999, 3584, 2622, 11867, 2072, 5754, 17287, 9285, 2004, 2009, 3084, 26794, 5003, 8159, 2172, 19366, 2008, 2126, 1996, 5754, 17287, 9285, 2131, 2443, 1999, 19130, 4563, 2004, 2092], 4, 84], [[2794, 3500, 4713, 4783, 2319, 2005, 4526, 1998, 9530, 8873, 27390, 2075, 19130, 8663, 18209, 2247, 2007, 8285, 3225, 2009, 1998, 2036, 2108, 2583, 2000, 3945, 1996, 2465, 15069, 2005, 5847, 2000, 2022, 5361], 0, 34], [[2794, 3231, 2553, 1998, 7375, 2005], 3, 6], [[2794, 2393, 2121, 4725, 1998, 3231, 3572, 2000, 2265, 1996, 7584, 1997, 5144, 20346, 2015, 1998, 4471, 4230], 3, 18], [[9706, 3619, 9124, 5852], 3, 4], [[5815, 3988, 2490, 2005, 1037, 6038, 2278, 6364,

In [ ]:
import random
random.shuffle(comments_with_len)

In [ ]:
print(comments_with_len)

[[[28681, 2080, 2023, 13531, 2060, 5852, 3046, 2000, 2131, 9436, 1997, 2009], 3, 12], [[3231, 4226, 5657, 6305, 4877, 2323, 2036, 3231, 21947, 7690, 9307, 2099], 3, 12], [[19130, 2794, 3231, 4760, 1996, 3291, 1996, 5813, 2097, 12530, 2006, 12247], 3, 12], [[28681, 2080, 8081, 2033, 2101], 3, 5], [[1040, 10343, 3600, 7396, 27404, 2005, 1996, 5164, 7965, 13593, 2000, 4638, 1996, 3570, 1997, 5371, 2291, 1998, 2709, 6413, 3642, 5278, 1996, 6434, 5164, 2453, 3338, 3231, 3572, 2036, 3602, 2023, 2442, 2022, 1996, 2197, 2240, 1997, 1996, 3189], 3, 40], [[28681, 2080, 2069, 18584, 2235, 1998, 2312, 7696, 2065, 2057, 2031, 2070], 0, 12], [[2065, 2045, 2024, 4118, 11709, 2342, 2000, 1999, 6767, 3489, 1996, 3670, 2007, 2216, 11709, 2065, 2045, 2024, 3674, 4520, 1997, 4118, 11709, 2045, 2323, 2022, 2012, 2560, 2008, 2116, 9089, 2098, 11423, 2065, 2045, 2024, 2062, 9089, 2098, 11423, 2084, 4520, 1997, 4118, 11709, 2023, 2089, 2709, 23375, 10288, 20110, 3258, 2065, 2045, 2024, 2062, 4520, 1997, 4118,

In [ ]:
# sort by the len of comments
#comments_with_len.sort(key=lambda x: x[2])

remove the length attribute from all the comments

In [ ]:
sorted_comments_labels = [(comment_lab[0], comment_lab[1]) for comment_lab in comments_with_len]

In [ ]:
# only comment and label sorted 
print(sorted_comments_labels)

[([28681, 2080, 2023, 13531, 2060, 5852, 3046, 2000, 2131, 9436, 1997, 2009], 3), ([3231, 4226, 5657, 6305, 4877, 2323, 2036, 3231, 21947, 7690, 9307, 2099], 3), ([19130, 2794, 3231, 4760, 1996, 3291, 1996, 5813, 2097, 12530, 2006, 12247], 3), ([28681, 2080, 8081, 2033, 2101], 3), ([1040, 10343, 3600, 7396, 27404, 2005, 1996, 5164, 7965, 13593, 2000, 4638, 1996, 3570, 1997, 5371, 2291, 1998, 2709, 6413, 3642, 5278, 1996, 6434, 5164, 2453, 3338, 3231, 3572, 2036, 3602, 2023, 2442, 2022, 1996, 2197, 2240, 1997, 1996, 3189], 3), ([28681, 2080, 2069, 18584, 2235, 1998, 2312, 7696, 2065, 2057, 2031, 2070], 0), ([2065, 2045, 2024, 4118, 11709, 2342, 2000, 1999, 6767, 3489, 1996, 3670, 2007, 2216, 11709, 2065, 2045, 2024, 3674, 4520, 1997, 4118, 11709, 2045, 2323, 2022, 2012, 2560, 2008, 2116, 9089, 2098, 11423, 2065, 2045, 2024, 2062, 9089, 2098, 11423, 2084, 4520, 1997, 4118, 11709, 2023, 2089, 2709, 23375, 10288, 20110, 3258, 2065, 2045, 2024, 2062, 4520, 1997, 4118, 11709, 2084, 9089, 209

convert the sorted dataset into a TensorFlow 2.0-compliant input dataset shape.

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_comments_labels, output_types=(tf.int32, tf.int32))

In [ ]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

 Pad our dataset for each batch.

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 107), dtype=int32, numpy=
 array([[28681,  2080,  2023, ...,     0,     0,     0],
        [ 3231,  4226,  5657, ...,     0,     0,     0],
        [19130,  2794,  3231, ...,     0,     0,     0],
        ...,
        [28681,  2080,  8081, ...,     0,     0,     0],
        [ 2023,  2003, 20578, ...,     0,     0,     0],
        [28681,  2080,  5672, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 1, 3, 1, 0, 0, 0, 1, 1, 1, 1, 3, 1,
        1, 0, 1, 0, 3, 1, 1, 3, 1, 1], dtype=int32)>)

divide the dataset into test and training sets.

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_comments_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES //10 # 5= 20% , 10= 10%
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

                                                            

In [ ]:
#  import ktrain
#  from ktrain import text

#  (x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=comments,
#                                                                       y_train=y,
                                                                  
#                                                                       maxlen=512,
#                                                                       random_state=120,
#                                                                       val_pct=0.1,
#                                                                       preprocess_mode='bert')


In [ ]:
# train_data_x=tf.convert_to_tensor(x_train, np.float32)
# train_data_y=tf.convert_to_tensor(y_train,np.float32)

# test_data_x=tf.convert_to_tensor(x_test, np.float32)
# test_data_y=tf.convert_to_tensor(y_test,np.float32)



Creating the model

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=5,
                 dropout_rate=0.1,
                 training=True,
                 trainable=False,

                
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu",)
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        # self.cnn_layer4 = layers.Conv1D(filters=cnn_filters,
        #                                 kernel_size=5,
        #                                 padding="valid",
        #                                 activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        # l_4 = self.cnn_layer4(l)
        # l_4 = self.pool(l_4) 
        
        concatenated = tf.concat([l_1, l_2,l_3], axis=-1) # (self.batch_size, 3 * self.cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 50
CNN_FILTERS = 100
DNN_UNITS = 512
OUTPUT_CLASSES = 5

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced',classes=["Requirement","Design","Defect","Test","Documentation"],y=df.label)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 0.829041095890411,
 1: 0.5094276094276095,
 2: 1.4206572769953052,
 3: 0.9824675324675325,
 4: 9.16969696969697}

In [ ]:

# text_model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=5,
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)


In [ ]:
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer=optimizer,
                       metrics=["accuracy"])

In [ ]:
text_model.fit(train_data, epochs=10, verbose=1)

Epoch 1/10
44/44 [==============================] - 4s 62ms/step - loss: 1.4568 - accuracy: 0.3526
Epoch 2/10
44/44 [==============================] - 3s 58ms/step - loss: 1.1416 - accuracy: 0.5765
Epoch 3/10
44/44 [==============================] - 3s 58ms/step - loss: 0.6287 - accuracy: 0.7782
Epoch 4/10
44/44 [==============================] - 3s 59ms/step - loss: 0.3781 - accuracy: 0.9015
Epoch 5/10
44/44 [==============================] - 3s 57ms/step - loss: 0.1653 - accuracy: 0.9560
Epoch 6/10
44/44 [==============================] - 3s 57ms/step - loss: 0.0834 - accuracy: 0.9747
Epoch 7/10
44/44 [==============================] - 3s 58ms/step - loss: 0.0754 - accuracy: 0.9822
Epoch 8/10
44/44 [==============================] - 3s 57ms/step - loss: 0.0450 - accuracy: 0.9907
Epoch 9/10
44/44 [==============================] - 3s 59ms/step - loss: 0.0239 - accuracy: 0.9952
Epoch 10/10
44/44 [==============================] - 3s 58ms/step - loss: 0.0184 - accuracy: 0.9952


In [ ]:
from sklearn.metrics import classification_report

results = text_model.evaluate(test_data)
print(results)



4/4 [==============================] - 0s 40ms/step - loss: 1.1928 - accuracy: 0.7734
[1.1928471326828003, 0.7734375]


In [ ]:
results = text_model.evaluate(train_data)
print(results)

44/44 [==============================] - 1s 20ms/step - loss: 0.0056 - accuracy: 0.9986
[0.005624810699373484, 0.9985559582710266]
